# Setup & Training Script
To be run after new videos are added

Based on the article [Guide to Build Video Classification Model](https://www.analyticsvidhya.com/blog/2019/09/step-by-step-deep-learning-tutorial-video-classification-python/)


In [1]:
!pip3 install opencv-python

### Importing Libraries

In [2]:
import os, os.path
import cv2  # for caputring videos
import math # for mathematical operations
import matplotlib.pyplot as plt # for plotting the images
%matplotlib inline
import pandas as pd
import numpy as np # for mathematical operations
from keras.utils import np_utils
from numpy import genfromtxt
from skimage.transform import resize # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

import keras
from keras.models import Sequential, load_model
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.applications.vgg16 import preprocess_input

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from skimage.transform import resize 
from glob import glob
import os, os.path

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

 # Turn the csv files into dictionaries
1. Opens and converts csv file
2. Gets tags
3. Gets points for each frame
4. Reshapes each frame array
5. Groups frames by three

In [3]:
# going into folder with training data
path="dataPoints_training/"

# creating a pandas dataframe 
train = pd.DataFrame()

# going into individual files
for filename in glob(os.path.join(path, '*.csv')):
    # go into each file and add all of the data into the train dataframe
    file = filename.split("/")[1]  # gets individual file name
    train = pd.read_csv(path + file, header=None)  # adds data to dataframe

    # creating empty dictionary named trainingFrames_FILENAME
    locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])] = {}
    # creating empty dictionary named trainingCombo_FILENAME
    locals()['trainingCombo{}'.format("_" + file.split(".csv")[0])] = {}
    
    #getting tag for each set of points from name of file
    tag =filename.split("/")[1].split("_")[3].split(".csv")[0]
    # creating tag key for each trainingCombo dictionary and adding tag value
    locals()['trainingCombo{}'.format("_" + file.split(".csv")[0])]["tag"] = tag

    # number of points per frame
    n = 12
    
    # create array for each frame and adds them to dictionary
    for i in range(len(train) // 12):
        # grouping each frame and making one big array (with 12 x,y points) aka grouping 12 arrays into one array
        data = train.to_numpy()[i * n:(i + 1) * n]
        # deleting frame number from above array
        data = np.delete(data, 0, 1)
        # reshaping each frame array into 24, 1
        data = np.reshape(data, (24, 1))
        # creating tag with name of number of frame and adding above array as value into trainingFrames
        locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)] = data
        

    # goes into each trainingFrame dictionary
    for i in locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]:
        # if the point (???) is 
        if int(i) < (len(locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]))-3:
            # counters for one and two above current frame respectively
            j = int(i)+1
            k = int(i)+2
            
            # creating local varriables that store current frame array and the two following
            combine = locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)]
            combineTwo = locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(j)]
            combineThree = locals()['trainingFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(k)]
            
            # appending current frame with the following two frames
            combine = np.append(combine, combineTwo, axis=1)
            combine = np.append(combine, combineThree, axis=1)
            
            # creating tag with name of number of frame and adding above combo into trainingCombo
            locals()['trainingCombo{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)] = combine  

# Do the same for test videos

In [4]:
path="dataPoints_test/"

test = pd.DataFrame()

for filename in glob(os.path.join(path, '*.csv')):    
    file = filename.split("/")[1]
    test = pd.read_csv(path + file, header=None)
    locals()['testFrames{}'.format("_" + file.split(".csv")[0])] = {}
    locals()['testCombo{}'.format("_" + file.split(".csv")[0])] = {}
        
    size = len(test[0]) // 12
    
    for i in range((len(test) + 12 - 1) // 12):
        data = test.to_numpy()[i * n:(i + 1) * n]
        data = np.delete(data, 0, 1)
        data = np.reshape(data, (24, 1))
        locals()['testFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)] = data

    for i in locals()['testFrames{}'.format("_" + file.split(".csv")[0])]:                
        if int(i) < len(locals()['testFrames{}'.format("_" + file.split(".csv")[0])])-3:
            j = int(i)+1
            k = int(i)+2
            
            combine = locals()['testFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)]
            combineTwo = locals()['testFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(j)]
            combineThree = locals()['testFrames{}'.format("_" + file.split(".csv")[0])]['{}'.format(k)]
            
            combine = np.append(combine, combineTwo, axis=1)
            combine = np.append(combine, combineThree, axis=1)
            
            locals()['testCombo{}'.format("_" + file.split(".csv")[0])]['{}'.format(i)] = combine     

# Put all the training frames into a dummy thicc array and make them tags numerical

In [5]:
# going into folder with training data
path="dataPoints_training/"

# create two empty arrays
points = []
tags = []

# going into individual files
for filename in glob(os.path.join(path, '*.csv')): 
    # go into each file and add all of the data into the train dataframe
    file = filename.split("/")[1]
    
    size = len(locals()['trainingCombo{}'.format("_" + file.split(".csv")[0])])
    
    for i in range(0, size-1):
        val = list(locals()['trainingCombo{}'.format("_" + file.split(".csv")[0])]["{}".format(i)])
        points.append(val)
    
    # add a tag for each frame in trainingCombo (??)
    for i in range(0, size-1):
        tags.append(locals()['trainingCombo{}'.format("_" + file.split(".csv")[0])]["tag"])
    
X = np.array(points)

tags = pd.DataFrame(tags)

# replace tags of "b" or "g" with 1 and 0. (1 represents bad and 0 represents good)
tags = tags.replace("b", 1)
tags = tags.replace("g", 0)


# Split models

In [6]:
# https://www.bitdegree.org/learn/train-test-split
# discuss more in depth with RW

X_train, X_valid, y_train, y_valid = train_test_split(X, tags, test_size=0.2)    # preparing the validation set

### Shape into single dimension

In [7]:
X.shape, X_train.shape, X_valid.shape

((249, 24, 3), (199, 24, 3), (50, 24, 3))

In [8]:
y_train.shape, y_valid.shape

((199, 1), (50, 1))

In [9]:
X_train.shape, X_valid.shape
X_train = X_train.reshape(199, 24*3)
X_valid = X_valid.reshape(50, 24*3)

In [10]:
#normalize me 
# X_train = X_train/X_train.max()      
# X_valid = X_valid/X_train.max()

# Build dat model

In [11]:
model = Sequential()
model.add(InputLayer((24*3,)))    # input layer
model.add(Dense(units=10, activation='sigmoid', input_shape=(24*3,))) # hidden layer
model.add(Dense(2, activation='softmax'))    # output layer

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                730       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 22        
Total params: 752
Trainable params: 752
Non-trainable params: 0
_________________________________________________________________


# Weights file

In [13]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

# Compile model

In [14]:
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit model

In [15]:
model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid), callbacks=[mcp_save], batch_size=2)

Train on 199 samples, validate on 50 samples
Epoch 1/20
199/199 [==============================] - 0s 1ms/step - loss: 0.6853 - accuracy: 0.5678 - val_loss: 0.6179 - val_accuracy: 0.7600
Epoch 2/20
199/199 [==============================] - 0s 505us/step - loss: 0.5943 - accuracy: 0.7588 - val_loss: 0.5729 - val_accuracy: 0.7600
Epoch 3/20
199/199 [==============================] - 0s 539us/step - loss: 0.5644 - accuracy: 0.7588 - val_loss: 0.5575 - val_accuracy: 0.7600
Epoch 4/20
199/199 [==============================] - 0s 545us/step - loss: 0.5559 - accuracy: 0.7588 - val_loss: 0.5526 - val_accuracy: 0.7600
Epoch 5/20
199/199 [==============================] - 0s 523us/step - loss: 0.5538 - accuracy: 0.7588 - val_loss: 0.5514 - val_accuracy: 0.7600
Epoch 6/20
199/199 [==============================] - 0s 552us/step - loss: 0.5531 - accuracy: 0.7588 - val_loss: 0.5512 - val_accuracy: 0.7600
Epoch 7/20
199/199 [==============================] - 0s 673us/step - loss: 0.5531 - accuracy

## What does this show us?

- approx. 78.39% with epoch = 30
- approx. 73.87% with epoch = 20
- approx. 75.88% with epoch = 10

In [16]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((199, 72), (50, 72), (199, 1), (50, 1))

What we need:
       - Predicted Tags (y_valid)??
       - Actual Tags (y_train)??
What needs to be done: 
       - give me a number plz.

In [18]:
# fine-tuning the model architecture
model = Sequential()
model.add(InputLayer((24*3,)))    # input layer
model.add(Dense(units=10, activation='sigmoid', input_shape=(24*3,))) # hidden layer
model.add(Dense(2, activation='softmax'))    # output layer

In [27]:
model = load_model('weight.hdf5')
model.compile(loss='sparse_categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])


# creating two lists to store predicted and actual tags
predict = []
actual = y_valid.to_numpy()

predictionTags = model.predict_classes(X_valid)
print(predictionTags)
predict.append(y.columns.values[s.mode(predictionTags)[0][0]])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]


NameError: name 'y' is not defined

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100